In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir("drive/Othercomputers/pc/VF2pp-in-CUDA/")


Mounted at /content/drive


In [ ]:
os.listdir()

['LICENSE', 'README.md', 'data', 'tmp', 'src', '.git', '.gitignore', '.tips']

CUDA SETUP

In [2]:
!pip install nvcc4jupyter
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661206 sha256=66031e549dbcc1333870b26775423fa3507a1d40154a4efda73dadf0e8af2d9d
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


GPU TYPE

In [3]:
!nvidia-smi
!nvcc --version
%load_ext nvcc4jupyter

Thu Jul  4 10:04:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


GPU INFO

In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>

void deviceQuery()
{
  cudaDeviceProp prop;
  int nDevices=0, i;
  cudaError_t ierr;

  ierr = cudaGetDeviceCount(&nDevices);
  if (ierr != cudaSuccess) { printf("Sync error: %s\n", cudaGetErrorString(ierr)); }



  for( i = 0; i < nDevices; ++i )
  {
     ierr = cudaGetDeviceProperties(&prop, i);
     printf("Device number: %d\n", i);
     printf("  Device name: %s\n", prop.name);
     printf("  Compute capability: %d.%d\n\n", prop.major, prop.minor);

     printf("  Clock Rate: %d kHz\n", prop.clockRate);
     printf("  Total SMs: %d \n", prop.multiProcessorCount);
     printf("  Shared Memory Per SM: %lu bytes\n", prop.sharedMemPerMultiprocessor);
     printf("  Registers Per SM: %d 32-bit\n", prop.regsPerMultiprocessor);
     printf("  Max threads per SM: %d\n", prop.maxThreadsPerMultiProcessor);
     printf("  L2 Cache Size: %d bytes\n", prop.l2CacheSize);
     printf("  Total Global Memory: %lu bytes\n", prop.totalGlobalMem);
     printf("  Memory Clock Rate: %d kHz\n\n", prop.memoryClockRate);


     printf("  Max threads per block: %d\n", prop.maxThreadsPerBlock);
     printf("  Max threads in X-dimension of block: %d\n", prop.maxThreadsDim[0]);
     printf("  Max threads in Y-dimension of block: %d\n", prop.maxThreadsDim[1]);
     printf("  Max threads in Z-dimension of block: %d\n\n", prop.maxThreadsDim[2]);

     printf("  Max blocks in X-dimension of grid: %d\n", prop.maxGridSize[0]);
     printf("  Max blocks in Y-dimension of grid: %d\n", prop.maxGridSize[1]);
     printf("  Max blocks in Z-dimension of grid: %d\n\n", prop.maxGridSize[2]);

     printf("  Shared Memory Per Block: %lu bytes\n", prop.sharedMemPerBlock);
     printf("  Registers Per Block: %d 32-bit\n", prop.regsPerBlock);
     printf("  Warp size: %d\n\n", prop.warpSize);

  }
}

int main() {
    deviceQuery();
}

Device number: 0
  Device name: Tesla T4
  Compute capability: 7.5

  Clock Rate: 1590000 kHz
  Total SMs: 40 
  Shared Memory Per SM: 65536 bytes
  Registers Per SM: 65536 32-bit
  Max threads per SM: 1024
  L2 Cache Size: 4194304 bytes
  Total Global Memory: 15835660288 bytes
  Memory Clock Rate: 5001000 kHz

  Max threads per block: 1024
  Max threads in X-dimension of block: 1024
  Max threads in Y-dimension of block: 1024
  Max threads in Z-dimension of block: 64

  Max blocks in X-dimension of grid: 2147483647
  Max blocks in Y-dimension of grid: 65535
  Max blocks in Z-dimension of grid: 65535

  Shared Memory Per Block: 49152 bytes
  Registers Per Block: 65536 32-bit
  Warp size: 32




In [29]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <cuda_runtime.h>
#include <limits.h>
#include <string.h>

#define FILENAME_QUERY "data/graph_query_500.csv"
#define FILENAME_TARGET "data/graph_target_500.csv"
#define LABELS 10
#define INF 99999

#define CUDA_CHECK_ERROR(err)           \
    if (err != cudaSuccess) {            \
        printf("CUDA error: %s\n", cudaGetErrorString(err)); \
        printf("Error in file: %s, line: %i\n", __FILE__, __LINE__); \
        exit(EXIT_FAILURE);              \
    }

/***** STRUCTS *****/
typedef struct {
    int* matrix;
    int numVertices;
    int* nodesToLabel;
    int** labelToNodes;
    int* labelsCardinalities;
    int* degrees;
} Graph;

typedef struct {
    int *mapping1;  // mapping from query to target
    int *mapping2;  // mapping from target to query
    int *T1;        // Ti contains uncovered neighbors of covered nodes from Gi, i.e. nodes that are not in the mapping, but are neighbors of nodes that are.
    int *T2;
    int* T1_out;     //Ti_out contains all the nodes from Gi, that are neither in the mapping nor in Ti. Cioe nodi che non sono in mapping e non sono vicini di nodi coperti
    int* T2_out;
} State;

typedef struct {
    int vertex;
    int* candidates;
    int sizeCandidates;
    int candidateIndex;
} Info;

typedef struct StackNode {
    Info* info;
    struct StackNode* next;
} StackNode;

/***** GRAPH PROTOTYPES *****/
void initGraph(Graph*);
Graph* createGraph();
void addEdge(Graph*, int, int);
Graph* readGraph(char*);
void printGraph(Graph*);
void freeGraph(Graph*);
void setLabel(Graph*, int, int);

/***** STATE PROTOTYPES *****/
State* createState(Graph*, Graph*);
void freeState(State*);
void printState(State*, int);
bool isMappingFull(Graph*, State*);
void updateState(Graph*, Graph*, State*, int, int);
void restoreState(Graph*, Graph*, State*, int, int);

/***** VF2++ PROTOTYPES *****/
void vf2pp(Graph*, Graph*, State*);
bool checkGraphProperties(Graph*, Graph*);
bool checkSequenceDegree(int*, int*, int);
int compare(const void*, const void*);
int* ordering(Graph*, Graph*);
int* bfs(Graph*, int, int*);
int findRoot(int, int*, int*, int*);
void processDepth(int*, int*, int*, int*, int*, int*, int, int*, int*, int*, int*);
int* findCandidates(Graph*, Graph*, State*, int, int*);
bool cutISO(Graph*, Graph*, State*, int, int);

/***** STACK PROTOTYPES *****/
Info* createInfo(int* candidates, int sizeCandidates, int vertex);
StackNode* createStackNode(Info*);
void push(StackNode**, int*, int, int);
Info* pop(StackNode**);
bool isStackEmpty(StackNode*);
void freeStack(StackNode*);
void printStack(StackNode*);
void printInfo(Info*);
void freeInfo(Info*);
StackNode* createStack();
Info* peek(StackNode**);

int main() {
    // printf("%s\n", FILENAME_QUERY);
    Graph* g1 = readGraph(FILENAME_QUERY);
    // printf("%s\n", FILENAME_TARGET);
    Graph* g2 = readGraph(FILENAME_TARGET);
    State* s = createState(g1, g2);

    // printf("\n");
    // printState(s, g1->numVertices);
    // printf("\n");

    vf2pp(g1, g2, s);

     printf("Mapping\n");
     for(int i = 0; i < g1->numVertices; i++) {
         printf("%d -> %d\n", i, s->mapping1[i]);
     }

    freeGraph(g1);
    freeGraph(g2);
    freeState(s);

    return EXIT_SUCCESS;
}

/***** GRAPH FUNCTIONS *****/
void initGraph(Graph* g) {
    g->matrix = (int*)malloc(g->numVertices * g->numVertices * sizeof(int));
    g->nodesToLabel = (int*)malloc(g->numVertices * sizeof(int));
    g->labelsCardinalities = (int*)malloc(LABELS * sizeof(int));
    g->labelToNodes = (int**)malloc(LABELS * sizeof(int*));
    g->degrees = (int*)malloc(g->numVertices * sizeof(int));

    if (g->nodesToLabel == NULL || g->labelsCardinalities == NULL || g->labelToNodes == NULL || g->degrees == NULL || g->matrix == NULL) {
        printf("Error allocating memory in initGraph\n");
        exit(EXIT_FAILURE);
    }

    for (int vertex = 0; vertex < g->numVertices; vertex++) {
        g->nodesToLabel[vertex] = -1;
        g->degrees[vertex] = 0;

        for (int adjVertex = 0; adjVertex < g->numVertices; adjVertex++) {
            g->matrix[vertex * g->numVertices + adjVertex] = 0;
        }
    }

    for (int label = 0; label < LABELS; label++) {
        g->labelsCardinalities[label] = 0;
        g->labelToNodes[label] = (int*)malloc(g->numVertices * sizeof(int));
    }
}

void setLabel(Graph* g, int node, int label) {
    if (g->nodesToLabel[node] == -1) {
        g->nodesToLabel[node] = label;
        g->labelsCardinalities[label]++;
        g->labelToNodes[label][g->labelsCardinalities[label] - 1] = node;
    }
}

void addEdge(Graph* g, int src, int target) {
    g->matrix[src * g->numVertices + target] = 1;
    g->matrix[target * g->numVertices + src] = 1;
    g->degrees[src]++;
    g->degrees[target]++;
}

Graph* createGraph() {
    Graph* g = (Graph*)malloc(sizeof(Graph));

    if (g == NULL) {
        printf("Error allocating memory in createGraph\n");
        exit(EXIT_FAILURE);
    }

    g->matrix = NULL;
    g->numVertices = 0;
    g->nodesToLabel = NULL;
    g->labelsCardinalities = NULL;
    g->degrees = NULL;
    g->labelToNodes = NULL;
    return g;
}

Graph* readGraph(char* path) {
    int src, target, srcLabel, targetLabel;
    Graph* g = createGraph();

    FILE* f = fopen(path, "r");
    if (f == NULL) {
        printf("Error opening file\n");
        exit(EXIT_FAILURE);
    }

    char line[128];
    fgets(line, sizeof(line), f);
    sscanf(line, "%*s%*s%*s%d", &g->numVertices);
    fgets(line, sizeof(line), f); // skip the header

    initGraph(g);

    while (fgets(line, sizeof(line), f)) {
        sscanf(line, "%d,%d,%d,%d", &src, &target, &srcLabel, &targetLabel);
        addEdge(g, src, target);
        setLabel(g, src, srcLabel);
        setLabel(g, target, targetLabel);
    }

    fclose(f);

    for(int label = 0; label < LABELS; label++) {
        g->labelToNodes[label] = (int*)realloc(g->labelToNodes[label], g->labelsCardinalities[label] * sizeof(int));
    }

    return g;
}

void printGraph(Graph* g) {
    for (int i = 0; i < g->numVertices; i++) {
        for (int j = 0; j < g->numVertices; j++) {
            printf("%d ", g->matrix[i * g->numVertices + j]);
        }
        printf("\tVertex %d, label %d, degree %d\n", i, g->nodesToLabel[i], g->degrees[i]);
    }

    printf("\nCardinalities\n");
    for (int i = 0; i < LABELS; i++) {
        printf("Label %d: %d\n", i, g->labelsCardinalities[i]);
    }

    for(int i = 0; i < LABELS; i++) {
       printf("\nLabel %d\n", i);
       for(int j = 0; j < g->labelsCardinalities[i]; j++) {
           printf("%d ", g->labelToNodes[i][j]);
       }
    }
}

void freeGraph(Graph* g) {
    for(int i = 0; i < LABELS; i++) {
        free(g->labelToNodes[i]);
    }
    free(g->labelToNodes);
    free(g->matrix);
    free(g->nodesToLabel);
    free(g->labelsCardinalities);
    free(g->degrees);
    free(g);
    g = NULL;
}

/***** STATE FUNCTIONS *****/
State* createState(Graph* g1, Graph* g2) {
    State* s = (State*)malloc(sizeof(State));

    s->mapping1 = (int*)malloc(g1->numVertices * sizeof(int));
    s->mapping2 = (int*)malloc(g2->numVertices * sizeof(int));
    s->T1 = (int*)malloc(g1->numVertices * sizeof(int));
    s->T2 = (int*)malloc(g2->numVertices * sizeof(int));
    s->T1_out = (int*)malloc(g1->numVertices * sizeof(int));
    s->T2_out = (int*)malloc(g2->numVertices * sizeof(int));

    if (s == NULL || s->mapping1 == NULL || s->mapping2 == NULL || s->T1 == NULL || s->T2 == NULL || s->T1_out == NULL || s->T2_out == NULL) {
        printf("Error allocating memory in createState\n");
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < g1->numVertices; i++) {
        s->mapping1[i] = -1;
        s->T1[i] = -1;
        s->T1_out[i] = 1;

        s->mapping2[i] = -1;
        s->T2[i] = -1;
        s->T2_out[i] = 1;
    }

    return s;
}

void freeState(State* s) {
    free(s->mapping1);
    free(s->mapping2);
    free(s->T1);
    free(s->T2);
    free(s->T1_out);
    free(s->T2_out);
    free(s);
    s = NULL;
}

void printState(State* s, int numVertices) {
    printf("Mapping 1\n");
    for (int i = 0; i < numVertices; i++) {
        printf("%d ", s->mapping1[i]);
    }

    printf("\nMapping 2\n");
    for (int i = 0; i < numVertices; i++) {
        printf("%d ", s->mapping2[i]);
    }

    printf("\nT1\n");
    for (int i = 0; i < numVertices; i++) {
        if(s->T1[i] != -1) {
            printf("%d ", i);
        }
        // printf("%d ", s->T1[i]);
    }

    printf("\nT2\n");
    for (int i = 0; i < numVertices; i++) {
        if(s->T2[i] != -1) {
            printf("%d ", i);
        }
        // printf("%d ", s->T2[i]);
    }

    printf("\nT1_out\n");
    for (int i = 0; i < numVertices; i++) {
        if(s->T1_out[i] != -1) {
            printf("%d ", i);
        }
        // printf("%d ", s->T1_out[i]);
    }

    printf("\nT2_out\n");
    for (int i = 0; i < numVertices; i++) {
        if(s->T2_out[i] != -1) {
            printf("%d ", i);
        }
        // printf("%d ", s->T2_out[i]);
    }
}

bool isMappingFull(Graph* g, State* state) {
    for(int i = 0; i < g->numVertices; i++) {
        if(state->mapping1[i] == -1) {
            return false;
        }
    }
    return true;
}

__global__ void updateStateKernel(int* matrix, int V, int* mapping, int* T, int* T_out, int node) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if( idx < V) {
        if(matrix[node * V + idx] == 1 && mapping[idx] == -1) {
            T[idx] = 1;
            T_out[idx] = -1;
        }
    }

}

void updateState(Graph* g1, Graph* g2, State* state, int node, int candidate) {
    int* d_matrix1, *d_matrix2;
    int* d_mapping1, *d_mapping2;
    int* d_T1, *d_T1_out;
    int* d_T2, *d_T2_out;

    cudaStream_t stream1, stream2;
    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix1, g1->numVertices * g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1_out, g1->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix1, g1->matrix, g1->numVertices * g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping1, state->mapping1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1, state->T1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1_out, state->T1_out, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));

    int blockSize = 256;
    int gridSize = (g1->numVertices + blockSize - 1) / blockSize;

    updateStateKernel<<<gridSize, blockSize, 0, stream1>>>(d_matrix1, g1->numVertices, d_mapping1, d_T1, d_T1_out, node);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix2, g2->numVertices * g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2_out, g2->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix2, g2->matrix, g2->numVertices * g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping2, state->mapping2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2, state->T2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2_out, state->T2_out, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));

    gridSize = (g2->numVertices + blockSize - 1) / blockSize;

    updateStateKernel<<<gridSize, blockSize, 0, stream2>>>(d_matrix2, g2->numVertices, d_mapping2, d_T2, d_T2_out, candidate);

    cudaStreamSynchronize(stream1);

    CUDA_CHECK_ERROR(cudaMemcpy(state->T1, d_T1, g1->numVertices * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(state->T1_out, d_T1_out, g1->numVertices * sizeof(int), cudaMemcpyDeviceToHost));

    cudaStreamDestroy(stream1);
    cudaFree(d_matrix1);
    cudaFree(d_mapping1);
    cudaFree(d_T1);
    cudaFree(d_T1_out);

    state->T1[node] = -1;
    state->T1_out[node] = -1;

    cudaStreamSynchronize(stream2);

    CUDA_CHECK_ERROR(cudaMemcpy(state->T2, d_T2, g2->numVertices * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(state->T2_out, d_T2_out, g2->numVertices * sizeof(int), cudaMemcpyDeviceToHost));

    cudaStreamDestroy(stream2);
    cudaFree(d_matrix2);
    cudaFree(d_mapping2);
    cudaFree(d_T2);
    cudaFree(d_T2_out);

    state->T2[candidate] = -1;
    state->T2_out[candidate] = -1;
}

__global__ void restoreStateKernel(int* matrix, int V, int node, int* T, int* T_out, int* mapping) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= V) {
        return;
    }

    int isAdded = 0;

    if(matrix[node * V + idx] == 1) {

        if(mapping[idx] != -1) {
            atomicExch(&T[node], 1);
            isAdded = 1;
        }
        else {
            int hasCoveredNeighbor = 0;
            for(int adjVertex2 = 0; adjVertex2 < V; adjVertex2++) {
                if(matrix[idx * V + adjVertex2] == 1 && mapping[adjVertex2] != -1) {
                    hasCoveredNeighbor = 1;
                    break;
                }
            }

            if(hasCoveredNeighbor == 0) {
                T[idx] = -1;
                T_out[idx] = 1;
            }
        }
    }

    if(isAdded == 0) {
        atomicExch(&T_out[node], 1);
    }
}

void restoreState(Graph* g1, Graph* g2, State* state, int node, int candidate) {
    int* d_matrix1, *d_matrix2;
    int* d_mapping1, *d_mapping2;
    int* d_T1, *d_T1_out;
    int* d_T2, *d_T2_out;

    cudaStream_t stream1, stream2;
    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix1, g1->numVertices * g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1_out, g1->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix1, g1->matrix, g1->numVertices * g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping1, state->mapping1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1, state->T1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1_out, state->T1_out, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));

    int blockSize = 256;
    int gridSize = (g1->numVertices + blockSize - 1) / blockSize;

    restoreStateKernel<<<gridSize, blockSize, 0, stream1>>>(d_matrix1, g1->numVertices, node, d_T1, d_T1_out, d_mapping1);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix2, g2->numVertices * g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2_out, g2->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix2, g2->matrix, g2->numVertices * g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping2, state->mapping2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2, state->T2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2_out, state->T2_out, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));

    gridSize = (g2->numVertices + blockSize - 1) / blockSize;

    restoreStateKernel<<<gridSize, blockSize, 0, stream2>>>(d_matrix2, g2->numVertices, candidate, d_T2, d_T2_out, d_mapping2);

    cudaStreamSynchronize(stream1);

    CUDA_CHECK_ERROR(cudaMemcpy(state->T1, d_T1, g1->numVertices * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(state->T1_out, d_T1_out, g1->numVertices * sizeof(int), cudaMemcpyDeviceToHost));

    cudaStreamDestroy(stream1);
    cudaFree(d_matrix1);
    cudaFree(d_mapping1);
    cudaFree(d_T1);
    cudaFree(d_T1_out);

    cudaStreamSynchronize(stream2);

    CUDA_CHECK_ERROR(cudaMemcpy(state->T2, d_T2, g2->numVertices * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(state->T2_out, d_T2_out, g2->numVertices * sizeof(int), cudaMemcpyDeviceToHost));

    cudaStreamDestroy(stream2);
    cudaFree(d_matrix2);
    cudaFree(d_mapping2);
    cudaFree(d_T2);
    cudaFree(d_T2_out);
}

/***** VF2++ FUNCTIONS *****/
bool checkGraphProperties(Graph* g1, Graph* g2) {
    if (g1->numVertices != g2->numVertices || g1->numVertices == 0 || g2->numVertices == 0) {
        return false;
    }

    if (!checkSequenceDegree(g1->degrees, g2->degrees, g1->numVertices)) {
        return false;
    }

    for(int i = 0; i < LABELS; i++) {
        if (g1->labelsCardinalities[i] != g2->labelsCardinalities[i]) {
            return false;
        }
    }

    return true;
}

bool checkSequenceDegree(int* degree1, int* degree2, int size) {
    int* tmp1 = (int*)malloc(size * sizeof(int));
    int* tmp2 = (int*)malloc(size * sizeof(int));

    memcpy(tmp1, degree1, size * sizeof(int));
    memcpy(tmp2, degree2, size * sizeof(int));

    qsort(tmp1, size, sizeof(int), compare);
    qsort(tmp2, size, sizeof(int), compare);
   
    bool ret = true;
    for (int i = 0; i < size; i++) {
        if (tmp1[i] != tmp2[i]) {
            ret = false;
            break;
        }
    }

    free(tmp1);
    free(tmp2);
    return ret;
}

int compare(const void* a, const void* b) {
    return (*(int*)a - *(int*)b);
}

__global__ void bfsKernel(int* matrix, int V, int* levels, int* d_done, int depth) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    __shared__ int s_done;
    extern __shared__ int s_levels[];

    if(threadIdx.x == 0) {
        s_done = 0;
    }

    // it copies the whole levels array in shared memory
    // example: thread 0 loads levels[0] and levels[4] if blockSize is 4
    for(int i = threadIdx.x; i < V; i += blockDim.x) {
      s_levels[i] = levels[i];
    }

    __syncthreads();

    // levels is used as visited too
    if(idx < V && s_levels[idx] == depth) {    // it blocks all thread with size greater than V and all threads not at the current depth

        for(int adjVertex = 0; adjVertex < V; adjVertex++) {
            if(matrix[idx * V + adjVertex] == 1 && s_levels[adjVertex] == -1) {
                atomicExch(&levels[adjVertex], depth + 1);
                s_done = 1;
            }
        }
    }

    __syncthreads();

    if(threadIdx.x == 0) {
        atomicExch(d_done, s_done);
    }
}

__global__ void maxRarityKernel(int V, int* d_labelRarity, int* d_nodesToLabel, int* d_maxRarity, int* d_is_good) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    extern __shared__ int s_data[];

    if(idx < V && d_is_good[idx]) {
        s_data[threadIdx.x] = d_labelRarity[d_nodesToLabel[idx]];
    } else {
        s_data[threadIdx.x] = INF;
    }

    __syncthreads();

    // parallel reduction: at each step, the block is halved and each thread computes the min of its value with the value of the other one at distance s in
    // the same block
    for(unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if(threadIdx.x < s) {
            s_data[threadIdx.x] = min(s_data[threadIdx.x], s_data[threadIdx.x + s]);   // At the end of the loop, the min value is in sdata[0]
        }
        __syncthreads();
    }

    // each thread 0 of each block computes the global min
    if(threadIdx.x == 0) {
        atomicMin(d_maxRarity, s_data[0]);
    }
}

__global__ void maxRarityFilterKernel(int V, int* d_labelRarity, int* d_nodesToLabel, int* d_maxRarity, int* d_is_good) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < V) {
        if(d_labelRarity[d_nodesToLabel[idx]] != *d_maxRarity) {
            d_is_good[idx] = 0;
        }
    }
}

__global__ void maxDegreeKernel(int V, int* d_degrees, int* d_maxDegree, int* d_is_good) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    extern __shared__ int s_data[];

    if(idx < V && d_is_good[idx]) {
        s_data[threadIdx.x] = d_degrees[idx];
    }
    else {
        s_data[threadIdx.x] = -INF;
    }

    __syncthreads();

    for(unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if(threadIdx.x < s) {
            s_data[threadIdx.x] = max(s_data[threadIdx.x], s_data[threadIdx.x + s]);
        }
        __syncthreads();
    }

    if(threadIdx.x == 0) {
        atomicMax(d_maxDegree, s_data[0]);
    }
}

__global__ void maxDegreeFilterKernel(int V, int* d_degrees, int* d_maxDegree, int* d_is_good) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < V) {
        if(d_degrees[idx] != *d_maxDegree) {
            d_is_good[idx] = 0;
        }
    }
}

__global__ void findNodeKernel(int V, int* is_good, int* d_node) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx < V && is_good[idx]) {
        atomicExch(d_node, idx);
    }
}

__global__ void initArrayKernel(int* d_array, int size, int value) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < size) {
        d_array[idx] = value;
    }
}

int findRoot(int V, int* d_labelRarity, int* d_nodesToLabel, int* d_degrees) {
    int* d_root, *d_is_good, *d_maxRarity, *d_maxDegree;
    int h_root = -1, h_maxRarity = INF, h_maxDegree = -INF;

    int blockSize = 256;
    int gridSize = (V + blockSize - 1) / blockSize;

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_root, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_is_good, V * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_maxRarity, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_maxDegree, sizeof(int)));

    initArrayKernel<<<gridSize, blockSize>>>(d_is_good, V, 1);
    CUDA_CHECK_ERROR(cudaMemcpy(d_maxRarity, &h_maxRarity, sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_maxDegree, &h_maxDegree, sizeof(int), cudaMemcpyHostToDevice));
    cudaDeviceSynchronize();

    size_t sharedMemSize = blockSize * sizeof(int); // each block has a shared memory of size blockSize

    maxRarityKernel<<<gridSize, blockSize, sharedMemSize>>>(V, d_labelRarity, d_nodesToLabel, d_maxRarity, d_is_good);
    cudaDeviceSynchronize();
    maxRarityFilterKernel<<<gridSize, blockSize>>>(V, d_labelRarity, d_nodesToLabel, d_maxRarity, d_is_good);
    cudaDeviceSynchronize();
    maxDegreeKernel<<<gridSize, blockSize, sharedMemSize>>>(V, d_degrees, d_maxDegree, d_is_good);
    cudaDeviceSynchronize();
    maxDegreeFilterKernel<<<gridSize, blockSize>>>(V, d_degrees, d_maxDegree, d_is_good);
    cudaDeviceSynchronize();
    findNodeKernel<<<gridSize, blockSize>>>(V, d_is_good, d_root);
    cudaDeviceSynchronize();

    CUDA_CHECK_ERROR(cudaMemcpy(&h_root, d_root, sizeof(int), cudaMemcpyDeviceToHost));

    cudaFree(d_maxDegree);
    cudaFree(d_is_good);
    cudaFree(d_maxRarity);
    cudaFree(d_root);
    return h_root;
}

__global__ void findLevelNodesKernel(int* levels, int depth, int* levelNodes, int V, int* levelSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    __shared__ int s_levelSize;

    if(threadIdx.x == 0) {
        s_levelSize = 0;
    }

    __syncthreads();

    if(idx < V && levels[idx] == depth) {
        atomicAdd(&s_levelSize, 1);
        levelNodes[idx] = 1;
    }

    __syncthreads();

    if(threadIdx.x == 0) {
        atomicAdd(levelSize, s_levelSize);
    }
}

int* ordering(Graph* g1, Graph* g2) {
    int* d_g1_matrix, *d_g1_nodesToLabel, *d_g1_degrees, *d_V1Unordered, *d_labelRarity, *d_connectivityG1;

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g1_matrix, g1->numVertices * g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g1_nodesToLabel, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g1_degrees, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_V1Unordered, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_labelRarity, LABELS * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_connectivityG1, g1->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpy(d_g1_matrix, g1->matrix, g1->numVertices * g1->numVertices * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_g1_nodesToLabel, g1->nodesToLabel, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_g1_degrees, g1->degrees, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_labelRarity, g1->labelsCardinalities, LABELS * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemset(d_V1Unordered, -1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMemset(d_connectivityG1, 0, g1->numVertices * sizeof(int)));

    int root = findRoot(g1->numVertices, d_labelRarity, d_g1_nodesToLabel, d_g1_degrees);

    int* levels = (int*)malloc(g1->numVertices * sizeof(int));
    int* order = (int*)malloc(g1->numVertices * sizeof(int));   // order of the nodes of g1
    int order_index = 0;

    if (order == NULL || levels == NULL) {
        printf("Error allocating memory in ordering\n");
        exit(EXIT_FAILURE);
    }

    for(int i = 0; i < g1->numVertices; i++) {
        levels[i] = -1;
    }

    int* d_levels, *d_done, *d_levelNodes, *d_levelSize;
    int depth = 0;
    int h_done;

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_levels, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_done, sizeof(int)));

    levels[root] = depth;

    CUDA_CHECK_ERROR(cudaMemcpy(d_levels, levels, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice));

    int blockSize = 256;
    int gridSize = (g1->numVertices + blockSize - 1) / blockSize;
    size_t sharedMemSize = sizeof(int) * g1->numVertices;

    do {
        h_done = 0;
        CUDA_CHECK_ERROR(cudaMemcpy(d_done, &h_done, sizeof(int), cudaMemcpyHostToDevice));

        bfsKernel<<<gridSize, blockSize, sharedMemSize>>>(d_g1_matrix, g1->numVertices, d_levels, d_done, depth);
        cudaDeviceSynchronize();

        CUDA_CHECK_ERROR(cudaMemcpy(&h_done, d_done, sizeof(int), cudaMemcpyDeviceToHost));
        depth++;
    } while(h_done);

    CUDA_CHECK_ERROR(cudaMemcpy(levels, d_levels, g1->numVertices*sizeof(int), cudaMemcpyDeviceToHost));

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_levelNodes, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_levelSize, sizeof(int)));

    gridSize = (g1->numVertices + blockSize - 1) / blockSize;
    for (int d = 0; d < depth; d++) {
        CUDA_CHECK_ERROR(cudaMemset(d_levelNodes, 0, g1->numVertices * sizeof(int)));
        CUDA_CHECK_ERROR(cudaMemset(d_levelSize, 0, sizeof(int)));

        findLevelNodesKernel<<<gridSize, blockSize, 0>>>(d_levels, d, d_levelNodes, g1->numVertices, d_levelSize);
        cudaDeviceSynchronize();
        processDepth(order, &order_index, d_connectivityG1, d_labelRarity, d_V1Unordered, d_levelNodes,
                    g1->numVertices, d_g1_degrees, d_g1_nodesToLabel, d_g1_matrix, d_levelSize);
    }

    free(levels);

    cudaFree(d_g1_matrix);
    cudaFree(d_g1_nodesToLabel);
    cudaFree(d_g1_degrees);
    cudaFree(d_V1Unordered);
    cudaFree(d_labelRarity);
    cudaFree(d_connectivityG1);
    cudaFree(d_levels);
    cudaFree(d_done);
    cudaFree(d_levelNodes);
    cudaFree(d_levelSize);

    return order;
}

__global__ void maxConnectivityKernel(int* d_connectivityG1, int* d_maxConnectivity, int* d_is_good, int V) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    extern __shared__ int s_data[];

    if(idx < V && d_is_good[idx]) { // d_is_good already contains the information about the nodes of the current level
        int conn = d_connectivityG1[idx];
        s_data[threadIdx.x] = conn;
    } else {
        s_data[threadIdx.x] = -INF;
    }

    __syncthreads();

    for(unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if(threadIdx.x < s) {
            s_data[threadIdx.x] = max(s_data[threadIdx.x], s_data[threadIdx.x + s]);
        }
        __syncthreads();
    }

    if(threadIdx.x == 0) {
        atomicMax(d_maxConnectivity, s_data[0]);
    }
}

__global__ void maxConnectivityFilterKernel(int* d_connectivityG1, int* d_maxConnectivity, int* d_is_good, int V) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < V) {
        if(d_connectivityG1[idx] != *d_maxConnectivity) {
            d_is_good[idx] = 0;
        }
    }
}

__global__ void unorderedFilterKernel(int* d_is_good, int* d_V1Unordered, int* d_levelNodes, int V) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx >= V) {
        return;
    }

    if(d_levelNodes[idx]) {
        if(d_V1Unordered[idx] == 1) {
            d_is_good[idx] = 0;
        }
    } else {
        d_is_good[idx] = 0;
    }
}

__global__ void updateConnKernel(int* matrix, int V, int* connectivity, int node) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < V) {
        if(matrix[node * V + idx] == 1) {
            atomicAdd(&connectivity[idx], 1);
        }
    }
}

void processDepth(int* order, int* order_index, int* d_connectivityG1, int* d_labelRarity, int* d_V1Unordered, int* d_levelNodes, int V,
                    int* d_degrees, int* d_nodesToLabel, int* d_matrix, int* d_levelSize) {

    int* d_is_good, *d_maxDegree, *d_maxRarity, *d_maxConnectivity, *d_nextNode;
    int h_nextNode, h_levelSize, h_maxRarity = INF, h_maxConnectivity = -INF, h_maxDegree = -INF;
    int* h_labelRarity = (int*)malloc(LABELS * sizeof(int));
    int* h_nodesToLabel = (int*)malloc(V * sizeof(int));
    int *h_V1Unordered = (int*)malloc(V * sizeof(int));

    CUDA_CHECK_ERROR(cudaMemcpy(&h_levelSize, d_levelSize, sizeof(int), cudaMemcpyDeviceToHost));

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_is_good, V * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_maxDegree, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_maxRarity, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_maxConnectivity, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_nextNode, sizeof(int)));

    int blockSize = 256;
    int gridSize = (V + blockSize - 1) / blockSize;
    size_t sharedMemSize = blockSize * sizeof(int);

    cudaStream_t stream;
    cudaStreamCreate(&stream);

    while(h_levelSize > 0) {
        initArrayKernel<<<gridSize, blockSize>>>(d_is_good, V, 1);
        cudaDeviceSynchronize();

        CUDA_CHECK_ERROR(cudaMemcpy(d_maxRarity, &h_maxRarity, sizeof(int), cudaMemcpyHostToDevice));
        CUDA_CHECK_ERROR(cudaMemcpy(d_maxDegree, &h_maxDegree, sizeof(int), cudaMemcpyHostToDevice));
        CUDA_CHECK_ERROR(cudaMemcpy(d_maxConnectivity, &h_maxConnectivity, sizeof(int), cudaMemcpyHostToDevice));

        unorderedFilterKernel<<<gridSize, blockSize>>>(d_is_good, d_V1Unordered, d_levelNodes, V);
        cudaDeviceSynchronize();
        maxConnectivityKernel<<<gridSize, blockSize, sharedMemSize>>>(d_connectivityG1, d_maxConnectivity, d_is_good, V);
        cudaDeviceSynchronize();
        maxConnectivityFilterKernel<<<gridSize, blockSize>>>(d_connectivityG1, d_maxConnectivity, d_is_good, V);
        cudaDeviceSynchronize();
        maxDegreeKernel<<<gridSize, blockSize, sharedMemSize>>>(V, d_degrees, d_maxDegree, d_is_good);
        cudaDeviceSynchronize();
        maxDegreeFilterKernel<<<gridSize, blockSize>>>(V, d_degrees, d_maxDegree, d_is_good);
        cudaDeviceSynchronize();
        maxRarityKernel<<<gridSize, blockSize, sharedMemSize>>>(V, d_labelRarity, d_nodesToLabel, d_maxRarity, d_is_good);
        cudaDeviceSynchronize();
        maxRarityFilterKernel<<<gridSize, blockSize>>>(V, d_labelRarity, d_nodesToLabel, d_maxRarity, d_is_good);
        cudaDeviceSynchronize();
        findNodeKernel<<<gridSize, blockSize>>>(V, d_is_good, d_nextNode);
        cudaDeviceSynchronize();

        CUDA_CHECK_ERROR(cudaMemcpy(&h_nextNode, d_nextNode, sizeof(int), cudaMemcpyDeviceToHost));

        updateConnKernel<<<gridSize, blockSize, 0, stream>>>(d_matrix, V, d_connectivityG1, h_nextNode);

        order[(*order_index)++] = h_nextNode;
        h_levelSize--;

        CUDA_CHECK_ERROR(cudaMemcpy(h_labelRarity, d_labelRarity, LABELS * sizeof(int), cudaMemcpyDeviceToHost));
        CUDA_CHECK_ERROR(cudaMemcpy(h_nodesToLabel, d_nodesToLabel, V * sizeof(int), cudaMemcpyDeviceToHost));
        CUDA_CHECK_ERROR(cudaMemcpy(h_V1Unordered, d_V1Unordered, V * sizeof(int), cudaMemcpyDeviceToHost));

        h_labelRarity[h_nodesToLabel[h_nextNode]]--;
        h_V1Unordered[h_nextNode] = 1;

        CUDA_CHECK_ERROR(cudaMemcpy(d_labelRarity, h_labelRarity, LABELS * sizeof(int), cudaMemcpyHostToDevice));
        CUDA_CHECK_ERROR(cudaMemcpy(d_nodesToLabel, h_nodesToLabel, V * sizeof(int), cudaMemcpyHostToDevice));
        CUDA_CHECK_ERROR(cudaMemcpy(d_V1Unordered, h_V1Unordered, V * sizeof(int), cudaMemcpyHostToDevice));

        cudaStreamSynchronize(stream);
    }

    cudaStreamDestroy(stream);
    cudaFree(d_maxDegree);
    cudaFree(d_maxRarity);
    cudaFree(d_maxConnectivity);
    cudaFree(d_nextNode);
    cudaFree(d_is_good);
}

__global__ void findCoveredNeighborsKernel(int* matrix1, int* mapping1, int node, int* coveredNeighbors, int* coveredNeighborsSize,
                                            int numVertices) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx >= numVertices)
        return;

    if(matrix1[node * numVertices + idx] == 1 && mapping1[idx] != -1) {
        int index = atomicAdd(coveredNeighborsSize, 1);
        coveredNeighbors[index] = idx;
    }
}

__global__ void findCandidatesKernel(int* coveredNeighborsSize, int g1_label, int maxSizeCandidates,
                                    int* g2_vertexList, int g1_degree, int* g2_degrees, int* T2_out, int* mapping2, int* candidates, int* candidateSize,
                                    int g2_numVertices, int* commonNodes, int* coveredNeighbors, int* g2_matrix, int* mapping1, int* g2_nodesToLabel) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(*coveredNeighborsSize == 0) {

        if(idx < maxSizeCandidates) {
            int vertex = g2_vertexList[idx];  // g2_labelToNodes[label]
            if(g2_degrees[vertex] == g1_degree && T2_out[vertex] == 1 && mapping2[vertex] != 1) {
                int index = atomicAdd(candidateSize, 1);
                candidates[index] = vertex;
            }
        }
    }
    else {

        if(idx < g2_numVertices) {
           commonNodes[idx] = 1;

            for (int i = 0; i < *coveredNeighborsSize; i++) {
                int nbrG1 = coveredNeighbors[i];
                int mappedG2 = mapping1[nbrG1];
                if (g2_matrix[mappedG2 * g2_numVertices + idx] == 0) {
                    commonNodes[idx] = 0;
                }
            }

            if (commonNodes[idx] && mapping2[idx] == 1) {
                commonNodes[idx] = 0;
            }

            if (commonNodes[idx] && g2_degrees[idx] != g1_degree) {
                commonNodes[idx] = 0;
            }

            if (commonNodes[idx] && g2_nodesToLabel[idx] != g1_label) {
                commonNodes[idx] = 0;
            }

            if (commonNodes[idx] == 1) {
                int index = atomicAdd(candidateSize, 1);
                candidates[index] = idx;
            }
        }
    }
}

int* findCandidates(Graph* g1, Graph* g2, State* state, int node, int* sizeCandidates) {
    // output
    int* d_coveredNeighbors, *d_coveredNeighborsSize;
    int coveredNeighborsSize = 0;

    // input
    int* d_matrix1, *d_mapping1;

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_coveredNeighbors, g1->degrees[node] * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_coveredNeighborsSize, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix1, g1->numVertices * g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping1, g1->numVertices * sizeof(int)));

    cudaStream_t stream1, stream2;
    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix1, g1->matrix, g1->numVertices * g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_coveredNeighborsSize, &coveredNeighborsSize, sizeof(int), cudaMemcpyHostToDevice, stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping1, state->mapping1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));

    int blockSize = 256;
    int gridSize = (g1->numVertices + blockSize - 1) / blockSize;

    findCoveredNeighborsKernel<<<gridSize, blockSize, 0, stream1>>>(d_matrix1, d_mapping1, node, d_coveredNeighbors, d_coveredNeighborsSize,
        g1->numVertices);

    int g1_label = g1->nodesToLabel[node];
    int maxSizeCandidates = g2->labelsCardinalities[g1_label];
    int g1_degree = g1->degrees[node];

    int* d_g2_vertexList, *d_g2_degrees, *d_g2_matrix, *d_g2_nodesToLabel;
    int* d_T2_out, *d_mapping2;
    int* d_candidates, *d_candidateSize, *d_commonNodes;
    int* candidates;
    int candidateSize = 0;
    
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g2_vertexList, maxSizeCandidates * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g2_degrees, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2_out, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_mapping2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_candidates, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_candidateSize, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_commonNodes, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g2_matrix, g2->numVertices * g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g2_nodesToLabel, g2->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g2_vertexList, g2->labelToNodes[g1_label], maxSizeCandidates * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g2_degrees, g2->degrees, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2_out, state->T2_out, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_mapping2, state->mapping2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_candidateSize, &candidateSize, sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g2_matrix, g2->matrix, g2->numVertices * g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g2_nodesToLabel, g2->nodesToLabel, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));

    cudaStreamSynchronize(stream1);

    blockSize = 256;
    gridSize = (g2->numVertices + blockSize - 1) / blockSize;
    findCandidatesKernel<<<gridSize, blockSize, 0, stream2>>>(d_coveredNeighborsSize, g1_label, maxSizeCandidates, d_g2_vertexList, g1_degree,
        d_g2_degrees, d_T2_out, d_mapping2, d_candidates, d_candidateSize, g2->numVertices, d_commonNodes, d_coveredNeighbors,
        d_g2_matrix, d_mapping1, d_g2_nodesToLabel);

    cudaStreamSynchronize(stream2);

    candidates = (int*)malloc(maxSizeCandidates * sizeof(int));
    CUDA_CHECK_ERROR(cudaMemcpy(candidates, d_candidates, maxSizeCandidates * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(&candidateSize, d_candidateSize, sizeof(int), cudaMemcpyDeviceToHost));
    *sizeCandidates = candidateSize;

    cudaStreamDestroy(stream1);
    cudaStreamDestroy(stream2);
    cudaFree(d_coveredNeighbors);
    cudaFree(d_coveredNeighborsSize);
    cudaFree(d_matrix1);
    cudaFree(d_mapping1);

    cudaFree(d_g2_vertexList);
    cudaFree(d_g2_degrees);
    cudaFree(d_T2_out);
    cudaFree(d_mapping2);
    cudaFree(d_candidates);
    cudaFree(d_candidateSize);
    cudaFree(d_commonNodes);
    cudaFree(d_g2_matrix);
    cudaFree(d_g2_nodesToLabel);

    return candidates;
}


void vf2pp(Graph* g1, Graph* g2, State* state) {
    if (!checkGraphProperties(g1, g2)) {
        return;
    }

    int* order = ordering(g1, g2);

    //printf("Order:\t");
    //for(int i = 0; i < g1->numVertices; i++) {
    //    printf("%d ", order[i]);
    //}
    //printf("\n");

    int sizeCandidates = 0;
    int* candidates = findCandidates(g1, g2, state, order[0], &sizeCandidates);

    StackNode* stack = createStack();
    push(&stack, candidates, sizeCandidates, order[0]);

    int matchingNode = 1;
    while (!isStackEmpty(stack)) {
        Info* info = peek(&stack);
        bool isMatch = false;

        // printInfo(info);

        for(int i = info->candidateIndex; i < info->sizeCandidates; i++) {
            int candidate = info->candidates[i];
            info->candidateIndex = i + 1;

            int ret = cutISO(g1, g2, state, info->vertex, candidate);

            // printf("CutISO: %d\n", ret);
            // printf("\n");

            if(!ret) {

                // printf("\nMatch %d -> %d\n", info->vertex, candidate);

                state->mapping1[info->vertex] = candidate;
                state->mapping2[candidate] = info->vertex;

                if(isMappingFull(g1, state)) {
                    freeStack(stack);
                    free(order);
                    printf("Graphs are isomorphic\n");
                    return;
                }

                updateState(g1, g2, state, info->vertex, candidate);
                candidates = findCandidates(g1, g2, state, order[matchingNode], &sizeCandidates);
                push(&stack, candidates, sizeCandidates, order[matchingNode]);
                matchingNode++;
                isMatch = true;
                break;
            }
        }

        // no more candidates
        if(!isMatch) {
            Info* tmp = pop(&stack);
            freeInfo(tmp);
            matchingNode--;

            // backtracking
            if(!isStackEmpty(stack)) {
                Info* prevInfo = peek(&stack);
                int candidate = state->mapping1[prevInfo->vertex];
                state->mapping1[prevInfo->vertex] = -1;
                state->mapping2[candidate] = -1;
                restoreState(g1, g2, state, prevInfo->vertex, candidate);
            }
        }
    }
    free(order);
    freeStack(stack);
}

__global__ void findNeighborsKernel(int* matrix, int node, int* neighbors, int* size, int numVertices) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < numVertices) {
        if(matrix[node * numVertices + idx] == 1) {
            int index = atomicAdd(size, 1);
            neighbors[index] = idx;
        }
    }
}

__global__ void checkLabelsKernel(int* neighbors1, int* neighbors2, int* nbrSize1, int* nbrSize2, int* labelsNbr, int numVertices,
    int* g1_nodesToLabel, int* g2_nodesToLabel, int* d_result) {   // idx is the id of the thread in the grid

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < *nbrSize1) {
        int nbr1 = neighbors1[idx];
        int labelNbr1 = g1_nodesToLabel[nbr1];
        bool found = false;

        for(int i = 0; i < *nbrSize2; i++) {
            int nbr2 = neighbors2[i];
            if(labelNbr1 == g2_nodesToLabel[nbr2]) {
                found = true;
                labelsNbr[labelNbr1] = 1;
                break;
            }
        }

        if(!found) {
            atomicExch(d_result, 0);   // d_result is initialized to 1 by default
        }
    }
}

__global__ void findNodesOfLabelKernel(int* neighbors, int* g_nodesToLabel, int label, int maxSize, int* size, int* nodes) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;    

    if(idx < maxSize) {
        int vertex = neighbors[idx];

        if(g_nodesToLabel[vertex] == label) {
            int index = atomicAdd(size, 1);   // atomicAdd returns the index respect to global memory (so can't be used shared memory)
            nodes[index] = vertex;
        }
    }
}

__global__ void intersectionCountKernel(int* nodes, int* size, int* stateSet, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;    // idx is the id of the thread in the grid among all threads
    __shared__ int localCount;  // each block has own localCount variable (shared memory)

    if(threadIdx.x == 0) {   // only one thread in the block initializes the localCount
        localCount = 0;
    }

    __syncthreads();

    if(idx < *size) {
        int vertex = nodes[idx];

        if(stateSet[vertex] == 1) {
            atomicAdd(&localCount, 1);
        }
    }

    __syncthreads();

    if(threadIdx.x == 0) {  // only the thread with id 0 of each block updates the global size
        atomicAdd(count, localCount);
    }
}

bool cutISO(Graph* g1, Graph* g2, State* state, int node1, int node2){
    // output
    int* d_neighbors1, *d_neighbors2;
    int* d_nbrSize1, *d_nbrSize2;
    int nbrSize1, nbrSize2;

    // input
    int* d_matrix1, *d_matrix2;

    cudaStream_t stream1, stream2, stream3;
    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);
    cudaStreamCreate(&stream3);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_neighbors1, g1->degrees[node1] * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_nbrSize1, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix1, g1->numVertices * g1->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemsetAsync(d_nbrSize1, 0, sizeof(int), stream1));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix1, g1->matrix, g1->numVertices * g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream1));

    int blockSize = 256;
    int gridSize = (g1->numVertices + blockSize - 1) / blockSize;

    findNeighborsKernel<<<gridSize, blockSize, 0, stream1>>>(d_matrix1, node1, d_neighbors1, d_nbrSize1, g1->numVertices);

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_neighbors2, g2->degrees[node2] * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_nbrSize2, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_matrix2, g2->numVertices * g2->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemsetAsync(d_nbrSize2, 0, sizeof(int), stream2));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_matrix2, g2->matrix, g2->numVertices * g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream2));

    gridSize = (g2->numVertices + blockSize - 1) / blockSize;

    findNeighborsKernel<<<gridSize, blockSize, 0, stream2>>>(d_matrix2, node2, d_neighbors2, d_nbrSize2, g2->numVertices);

    // input
    int* d_labelsNbr, *d_g1_nodesToLabel, *d_g2_nodesToLabel;
    int* labelsNbr = (int*)malloc(LABELS * sizeof(int));

    // output
    int* d_result;
    int result;

    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_labelsNbr, LABELS * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_result, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g1_nodesToLabel, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_g2_nodesToLabel, g2->numVertices * sizeof(int)));

    CUDA_CHECK_ERROR(cudaMemsetAsync(d_labelsNbr, 0, LABELS * sizeof(int), stream3));
    CUDA_CHECK_ERROR(cudaMemsetAsync(d_result, 1, sizeof(int), stream3));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g1_nodesToLabel, g1->nodesToLabel, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream3));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_g2_nodesToLabel, g2->nodesToLabel, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream3));

    cudaStreamSynchronize(stream1);
    cudaStreamSynchronize(stream2);

    CUDA_CHECK_ERROR(cudaMemcpy(&nbrSize1, d_nbrSize1, sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(&nbrSize2, d_nbrSize2, sizeof(int), cudaMemcpyDeviceToHost));

    cudaFree(d_matrix1);
    cudaFree(d_matrix2);

    blockSize = 256;
    gridSize = (nbrSize1 + blockSize - 1) / blockSize;

    checkLabelsKernel<<<gridSize, blockSize, 0, stream3>>>(d_neighbors1, d_neighbors2, d_nbrSize1, d_nbrSize2,
        d_labelsNbr, g1->numVertices, d_g1_nodesToLabel, d_g2_nodesToLabel, d_result);

    cudaStreamSynchronize(stream3);

    CUDA_CHECK_ERROR(cudaMemcpy(&result, d_result, sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK_ERROR(cudaMemcpy(labelsNbr, d_labelsNbr, LABELS * sizeof(int), cudaMemcpyDeviceToHost));

    cudaFree(d_labelsNbr);
    cudaFree(d_result);

    if(result == 0) {
        cudaStreamDestroy(stream1);
        cudaStreamDestroy(stream2);
        cudaStreamDestroy(stream3);

        cudaFree(d_neighbors1);
        cudaFree(d_neighbors2);
        cudaFree(d_nbrSize1);
        cudaFree(d_nbrSize2);
        cudaFree(d_g1_nodesToLabel);
        cudaFree(d_g2_nodesToLabel);

        return true;
    }

    cudaStream_t stream4, stream5, stream6;
    cudaStreamCreate(&stream4);
    cudaStreamCreate(&stream5);
    cudaStreamCreate(&stream6);

    // istantiating variables on device stream1 and stream2
    int* d_nodes_g1, *d_nodes_g2;
    int* d_size_g1, *d_size_g2;
    int size1, size2;

    // stream 1
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_nodes_g1, nbrSize1 * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_size_g1, sizeof(int)));

    // stream 2
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_nodes_g2, nbrSize2 * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_size_g2, sizeof(int)));

    //instantiating variables on device stream3, stream4, stream5, stream6
    int* d_count3, *d_count4, *d_count5, *d_count6;
    int count3, count4, count5, count6;
    int* d_T1, *d_T2;
    int* d_T1_out, *d_T2_out;

    // stream 3
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_count3, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1, state->T1, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream3));

    // stream 4
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_count4, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2, state->T2, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream4));

    // stream 5
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_count5, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T1_out, g1->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T1_out, state->T1_out, g1->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream5));

    // stream 6
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_count6, sizeof(int)));
    CUDA_CHECK_ERROR(cudaMalloc((void**)&d_T2_out, g2->numVertices * sizeof(int)));
    CUDA_CHECK_ERROR(cudaMemcpyAsync(d_T2_out, state->T2_out, g2->numVertices * sizeof(int), cudaMemcpyHostToDevice, stream6));

    bool ret = false;
    for(int label = 0; label < LABELS; label++) {
        if(labelsNbr[label] == 1) {

            // stream 1
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_size_g1, 0, sizeof(int), stream1));

            gridSize = (nbrSize1 + blockSize - 1) / blockSize;

            findNodesOfLabelKernel<<<gridSize, blockSize, 0, stream1>>>(d_neighbors1, d_g1_nodesToLabel, label, nbrSize1, d_size_g1, d_nodes_g1);

            // stream 2
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_size_g2, 0, sizeof(int), stream2));

            gridSize = (nbrSize2 + blockSize - 1) / blockSize;

            findNodesOfLabelKernel<<<gridSize, blockSize, 0, stream2>>>(d_neighbors2, d_g2_nodesToLabel, label, nbrSize2, d_size_g2, d_nodes_g2);

            // stream 3
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_count3, 0, sizeof(int), stream3));

            // stream 4
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_count4, 0, sizeof(int), stream4));

            // stream 5
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_count5, 0, sizeof(int), stream5));

            // stream 6
            CUDA_CHECK_ERROR(cudaMemsetAsync(d_count6, 0, sizeof(int), stream6));

            cudaStreamSynchronize(stream1);
            cudaStreamSynchronize(stream2);

            CUDA_CHECK_ERROR(cudaMemcpy(&size1, d_size_g1, sizeof(int), cudaMemcpyDeviceToHost));
            CUDA_CHECK_ERROR(cudaMemcpy(&size2, d_size_g2, sizeof(int), cudaMemcpyDeviceToHost));

            gridSize = (size1 + blockSize - 1) / blockSize;
            intersectionCountKernel<<<gridSize, blockSize, 0, stream3>>>(d_nodes_g1, d_size_g1, d_T1, d_count3);

            gridSize = (size2 + blockSize - 1) / blockSize;
            intersectionCountKernel<<<gridSize, blockSize, 0, stream4>>>(d_nodes_g2, d_size_g2, d_T2, d_count4);

            gridSize = (size1 + blockSize - 1) / blockSize;
            intersectionCountKernel<<<gridSize, blockSize, 0, stream5>>>(d_nodes_g1, d_size_g1, d_T1_out, d_count5);

            gridSize = (size2 + blockSize - 1) / blockSize;
            intersectionCountKernel<<<gridSize, blockSize, 0, stream6>>>(d_nodes_g2, d_size_g2, d_T2_out, d_count6);

            cudaStreamSynchronize(stream3);
            CUDA_CHECK_ERROR(cudaMemcpy(&count3, d_count3, sizeof(int), cudaMemcpyDeviceToHost));

            cudaStreamSynchronize(stream4);
            CUDA_CHECK_ERROR(cudaMemcpy(&count4, d_count4, sizeof(int), cudaMemcpyDeviceToHost));

            cudaStreamSynchronize(stream5);
            CUDA_CHECK_ERROR(cudaMemcpy(&count5, d_count5, sizeof(int), cudaMemcpyDeviceToHost));

            cudaStreamSynchronize(stream6);
            CUDA_CHECK_ERROR(cudaMemcpy(&count6, d_count6, sizeof(int), cudaMemcpyDeviceToHost));

            if(count3 != count4 || count5 != count6) {
                ret = true;
                break;
            }
        }
    }
    cudaStreamDestroy(stream1);
    cudaStreamDestroy(stream2);
    cudaStreamDestroy(stream3);
    cudaStreamDestroy(stream4);
    cudaStreamDestroy(stream5);
    cudaStreamDestroy(stream6);

    cudaFree(d_neighbors1);
    cudaFree(d_neighbors2);
    cudaFree(d_nbrSize1);
    cudaFree(d_nbrSize2);
    cudaFree(d_g1_nodesToLabel);
    cudaFree(d_g2_nodesToLabel);

    cudaFree(d_count3);
    cudaFree(d_count4);
    cudaFree(d_count5);
    cudaFree(d_count6);

    cudaFree(d_T1);
    cudaFree(d_T2);
    cudaFree(d_T1_out);
    cudaFree(d_T2_out);

    cudaFree(d_nodes_g1);
    cudaFree(d_size_g1);
    cudaFree(d_nodes_g2);
    cudaFree(d_size_g2);

    return ret;
}

/***** STACK FUNCTIONS *****/
Info* createInfo(int* candidates, int sizeCandidates, int vertex) {
    Info* info = (Info*)malloc(sizeof(Info));
    if (info == NULL) {
        printf("Error allocating memory in createInfo\n");
        exit(EXIT_FAILURE);
    }
    info->vertex = vertex;
    info->candidates = candidates;
    info->sizeCandidates = sizeCandidates;
    info->candidateIndex = 0;
    return info;
}

StackNode* createStackNode(Info* info) {
    StackNode* node = (StackNode*)malloc(sizeof(StackNode));
    if (node == NULL) {
        printf("Error allocating memory in createStackNode\n");
        exit(EXIT_FAILURE);
    }
    node->info = info;
    node->next = NULL;
    return node;
}

void push(StackNode** top, int* candidates, int sizeCandidates, int vertex) {
    Info* info = createInfo(candidates, sizeCandidates, vertex);
    StackNode* node = createStackNode(info);
    node->next = *top;
    *top = node;
}

Info* pop(StackNode** top) {
    if (isStackEmpty(*top)) {
        printf("Stack is empty, cannot pop\n");
        return NULL;
    }
    StackNode* node = *top;
    Info* info = node->info;
    *top = node->next;
    free(node);
    return info;
}

bool isStackEmpty(StackNode* top) {
    return top == NULL;
}

void freeStack(StackNode* top) {
    while (!isStackEmpty(top)) {
        StackNode* node = top;
        top = top->next;
        freeInfo(node->info);
        free(node);
    }
}

void printStack(StackNode* top) {
    StackNode* current = top;
    while (current != NULL) {
        printInfo(current->info);
        current = current->next;
    }
}

void printInfo(Info* info) {
    printf("\nVertex: %d\n", info->vertex);
    printf("Index seen: %d\n", info->candidateIndex);
    printf("Candidates: ");
    for (int i = 0; i < info->sizeCandidates; i++) {
        printf("%d ", info->candidates[i]);
    }
    printf("\n");
}

void freeInfo(Info* info) {
    free(info->candidates);
    free(info);
}

StackNode* createStack() {
    return NULL;
}

Info* peek(StackNode** top) {
    return (*top)->info;
}

init is_good
1 1 1 1 1 1 1 1 1 1 
max rarity
1
maxrarity filter
1 1 1 1 0 1 0 1 0 0 
max degree
7
maxdegree filter
0 1 0 0 0 0 0 0 0 0 
next node
1

Levels at depth 0
0 1 0 0 0 0 0 0 0 0 

init is_good
1 1 1 1 1 1 1 1 1 1 
unoreder filter
0 1 0 0 0 0 0 0 0 0 
max conn
0
maxconn filter
0 1 0 0 0 0 0 0 0 0 
max degree
7
maxdegree filter
0 1 0 0 0 0 0 0 0 0 
max rarity
1
maxrarity filter
0 1 0 0 0 0 0 0 0 0 
next node
1
update conn
0 0 1 1 1 1 1 0 1 1 

Levels at depth 1
0 0 1 1 1 1 1 0 1 1 

init is_good
1 1 1 1 1 1 1 1 1 1 
unoreder filter
0 0 1 1 1 1 1 0 1 1 
max conn
1
maxconn filter
0 0 1 1 1 1 1 0 1 1 
max degree
7
maxdegree filter
0 0 0 0 0 0 0 0 0 1 
max rarity
2
maxrarity filter
0 0 0 0 0 0 0 0 0 1 
next node
9
update conn
1 1 2 1 2 1 2 1 2 1 

init is_good
1 1 1 1 1 1 1 1 1 1 
unoreder filter
0 0 1 1 1 1 1 0 1 0 
max conn
2
maxconn filter
0 0 1 0 1 0 1 0 1 0 
max degree
5
maxdegree filter
0 0 1 0 0 0 0 0 1 0 
max rarity
1
maxrarity filter
0 0 1 0 0 0 0 0 0 0 
next node
2
update 